In [5]:
from heapq import heappush, heappop
from collections import deque
import copy
WIN_LINES = [
    (0,1,2), (3,4,5), (6,7,8),  # rows
    (0,3,6), (1,4,7), (2,5,8),  # cols
    (0,4,8), (2,4,6)            # diagonals
]
def check_winner(board):
    """Return 'X' or 'O' if someone won, or 'D' for draw, or None for non-terminal."""
    for a,b,c in WIN_LINES:
        if board[a] != ' ' and board[a] == board[b] == board[c]:
            return board[a]
    if ' ' not in board:
        return 'D'
    return None
def available_moves(board):
    return [i for i,cell in enumerate(board) if cell == ' ']
def make_move(board, pos, player):
    lst = list(board)
    lst[pos] = player
    return tuple(lst)
def board_lines_info(board, player):
    """Return info for each winning line: (player_marks, opponent_marks, empty_count)"""
    opp = 'O' if player == 'X' else 'X'
    info = []
    for a,b,c in WIN_LINES:
        marks = [board[a], board[b], board[c]]
        player_marks = marks.count(player)
        opp_marks = marks.count(opp)
        empty = marks.count(' ')
        info.append((player_marks, opp_marks, empty))
    return info
def heuristic(board, player):
    """Optimistic heuristic: minimum number of *player moves* needed to complete any line that has no opponent mark.
       If no such line exists, return a large number.
    """
    info = board_lines_info(board, player)
    best = None
    for pm, om, empty in info:
        if om == 0:  # opponent not blocking this line
            needed = 3 - pm  # how many of player's marks needed to complete
            if best is None or needed < best:
                best = needed
    if best is None:
        return 100  # no available winning line (opponent blocks all), pessimistic large h
    return best
def a_star_find_forced_win(start_board, player, max_nodes=200000):
    """
    Attempt to find a sequence of plies (moves by both players) that leads `player` to a win.
    Returns a list of (player_to_move, move_pos, resulting_board) representing the path of plies,
    or None if not found in search limits.
    """
    start_turn = 'X' if start_board.count('X') == start_board.count('O') else 'O'  # who's turn
    # Each node in open set: (f, g, id, board, turn, path)
    # path: list of (turn, move_pos)
    counter = 0
    start_h = heuristic(start_board, player)
    start_g = 0
    start_f = start_g + start_h
    open_heap = []
    heappush(open_heap, (start_f, start_g, counter, start_board, start_turn, []))
    closed = set()
    nodes_explored = 0
    while open_heap and nodes_explored < max_nodes:
        f, g, _, board, turn, path = heappop(open_heap)
        nodes_explored += 1
        state_id = (board, turn)
        if state_id in closed:
            continue
        closed.add(state_id)


        winner = check_winner(board)
        if winner == player:
            # reconstruct full path of boards
            full = []
            cur = start_board
            for mv_turn, mv_pos in path:
                cur = make_move(cur, mv_pos, mv_turn)
                full.append((mv_turn, mv_pos, cur))
            return full  # found sequence leading to player's win
        if winner is not None:
            # terminal but not player's win, skip expansion
            continue
        # Expand children (all legal moves for `turn`)
        for mv in available_moves(board):
            new_board = make_move(board, mv, turn)
            next_turn = 'O' if turn == 'X' else 'X'
            new_path = path + [(turn, mv)]
            new_g = g + 1  # one more ply
            # heuristic always measures distance for `player`
            h = heuristic(new_board, player)
            new_f = new_g + h
            counter += 1
            heappush(open_heap, (new_f, new_g, counter, new_board, next_turn, new_path))
    # not found within limits
    return None
# Simple helper to pretty-print board
def pretty(board):
    rows = []
    for i in range(0,9,3):
        rows.append(' | '.join(board[i:i+3]))
    return '\n---------\n'.join(rows)
# Demonstration: try to find a forcing sequence for X on an empty board
start = tuple(' ' for _ in range(9))
solution = a_star_find_forced_win(start, 'X', max_nodes=100000)
print("Explored: A* attempted to find forced win for X from empty board.")
if solution is None:
    print("No forced win found within search limits (expected for optimal opponent play).")
else:
    print("Found a sequence of plies leading to X win:")
    cur = start
    print(pretty(cur))
    for turn, pos, board in solution:
        print(f"\nMove: {turn} -> {pos}\n{pretty(board)}")

Explored: A* attempted to find forced win for X from empty board.
Found a sequence of plies leading to X win:
  |   |  
---------
  |   |  
---------
  |   |  

Move: X -> 0
X |   |  
---------
  |   |  
---------
  |   |  

Move: O -> 1
X | O |  
---------
  |   |  
---------
  |   |  

Move: X -> 3
X | O |  
---------
X |   |  
---------
  |   |  

Move: O -> 2
X | O | O
---------
X |   |  
---------
  |   |  

Move: X -> 6
X | O | O
---------
X |   |  
---------
X |   |  
